# Spiking Convolutional Neural Network in Speck

In this section you will learn about Speck, a neuromorphic system that integrates a Dynamic Vision Sensor and a Spiking Convolutional Neural Network on device. In the following sections, you will learn how to interact with the Speck data, visualize them and how to train a network that can be loaded to Speck.

You will train an SNN to solve a simple task and load it in Speck. You will then be able to run the network on inference and visualize its response in real time.



## Speck device

Speck is a neuromorphic device with characteristics in hardware and Samna has developed the software that we'll see

### Hardware

Speck's [architecture](https://synsense-sys-int.gitlab.io/samna/models/speckSeries/summary.html) consist of a Dynamic Vision Sensor and a Spiking Convolutional Neural Network with the following characteristics

- DVS input of 128x128 resolution
- 9 convolutional layers
- Implementation of Integrate and Fire neurons
- Leak through bias and external clock

### Software

Synsense provides software libraries to develop neural networks and interact with Speck.

- Communication with Speck relies on the [Samna](https://synsense-sys-int.gitlab.io/samna/index.html) library.
- Simulator [sinabs-dynapcnn](https://synsense.gitlab.io/sinabs-dynapcnn/) used to train models offline and load them to Speck.

## Interaction with Speck

In this section, we will see how to compute using speck input data, we will visualize events and develop a network for edge detection that we'll load to speck.


### Task 0: Import libraries and data 

We will start our preparation by importing the libraries necessary and the data for our tasks

#### Task 0.1: Import libraries

We begin by importing the necessary libraries for our implementation, such as samna, sinabs, torch and libraries to help us fetch and vsualize our data.

In [ ]:
!pip install -q samna
!pip install -q sinabs

import samna
import torch
import pickle
import tqdm
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML
from torch.utils.data import DataLoader, Dataset, random_split
from urllib.request import urlretrieve
import sinabs.layers as sl
from torch import nn
from sinabs.activation.surrogate_gradient_fn import PeriodicExponential
from operator import iconcat
from functools import reduce
import pickle

#### Task 0.2: Fetch data

Now we will fetch the data files that we will use for our next experiment.

You will notice that the format of the spikes is not the tensor format that has been previously discussed. What is this new format?

In [ ]:
data_file, _ = urlretrieve("https://github.com/ncskth/phd-course/raw/main/book/module4/edge_detect")
with open(data_file, 'rb') as f:
  data = pickle.load(f)

print(data[0])

### Task 1: Visualize DVS output

In this task you are asked to visualize your input data. Before working with the data, we should convert them in a friendly format (a tensor).

Note that we bin events every 10000μs (10ms). You will need this information when constructing the video from events.

In [ ]:
def events_to_channels(events, time_interval = 1000):
    frames = []
    index = 0
    current_frame = torch.zeros(2, 128, 128)
    for (t, y, x, p) in tqdm.tqdm(events.int()):
        if t // time_interval > index:
            frames.append(current_frame.clone())
            current_frame.fill_(0)
            index += 1
        current_frame[p, x, y] = +1
    return torch.stack(frames)


def Speck_spikes_to_events(sp_events):

  sp_events = reduce(iconcat, sp_events, [])
  events = torch.zeros((len(sp_events), 4))
  start = sp_events[0].timestamp

  for i, e in tqdm.tqdm(enumerate(sp_events), total=len(sp_events)):
    events[i][0] = e.timestamp - start
    events[i][1] = e.x
    events[i][2] = e.y
    events[i][3] = e.feature
  return events



# Convert Speck spikes to tensor

events = Speck_spikes_to_events(data)
events = events_to_channels(events, 10000)

Use the functions that are provided in the cell bellow to create a video of your input data.

In [ ]:
def animate_frames(frames, figure=None, interval: int = 20, **kwargs):
    if figure is None:
        figure, _ = plt.subplots(**kwargs)
    ax = figure.gca()

    image = ax.imshow(frames[0])  # .T)
    ax.set_axis_off()

    def animate(index):
        image.set_data(frames[index])  # .T)
        return image

    anim = FuncAnimation(figure, animate, frames=len(frames), interval=interval)
    video = anim.to_html5_video()
    html = HTML(video)
    display(html)
    plt.tight_layout()
    plt.close()


def events_to_frames(frames, polarity: bool = True):
    if len(frames.shape) == 3:
        frames = frames.unsqueeze(-1).repeat(1, 1, 1, 3)
    else:
        if not polarity:
            frames = frames.abs().sum(-1)
        elif polarity:
            frames = torch.concat([frames, torch.zeros(frames.shape[0], 1, *frames.shape[2:], device=frames.device)], dim=1).movedim(1, -1)
    frames = ((frames / frames.max()) * 255).int().clip(0, 255)
    return frames

# Visualize your input data

# ...

### Task 2: Create edge detector in sinabs

For this task you will have to create an edge detector. You will need to create a Spiking Convolutional Neural Network that detects vertical and horizontal edges.

Note that your neuros should be Integrate and Fire as Speck does not implement leak directly!

#### Task 2.1: Convolutional layer

In the following cell, you have an example of an edge detector. Use this as your kernel to develop your network. What should the shape of the convolutional layer's weights be?

In [ ]:
kernel_size = 9
gaussian = torch.sigmoid(torch.linspace(-10, 10, kernel_size + 1))
kernel = (gaussian.diff()-0.14).repeat(kernel_size, 1)
plt.imshow(kernel)
plt.colorbar()


#### Task 2.2: Network implementation

For this task, you will have to implement and run your network using the sinabs library. This library will allow you to transfer your network to the Speck chip. You can use `sl.IAFSqueeze()` to implement the integration of spikes.

Note: The biases should be set to None!

In [95]:
snn_bptt = nn.Sequential(
    # [2, 128, 128] -> [2, 64, 64]
    # Fill in your convolutional layer here. Note: Bias should be None!
    sl.IAFSqueeze(batch_size=1, min_v_mem=-1.0),
    )

out = snn_bptt(events)

# Every positive output of the network is a spike produced by the network. We threshold with out > 0 to normalize the output to 0 or 1 (spike or no spike) at each frame
print(torch.sum(out - out.int()))
out = (out > 0).float()

tensor(0.)


In [ ]:
animate_frames(events_to_frames(out[:400], polarity=True))

#### Task 2.3: Visualize the output

Using the provided function above, visualize the output. What do you observe? How can you suppress the noise?

Speck neurons can implement [leak](https://synsense.gitlab.io/sinabs-dynapcnn/getting_started/notebooks/leak_neuron.html) through their bias.


In [ ]:
# Visualize the output

# ...